##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Capas personalizadas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a> </td>
</table>

Como API de alto nivel para construir redes neuronales, recomendamos usar `tf.keras`. Sin embargo, la mayoría de las API de TensorFlow son utilizables con eager execution.


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.config.list_physical_devices('GPU'))

## Capas: grupos comunes de operaciones útiles

Cuando escriba código para modelos de aprendizaje automático, normalmente querrá operar a un nivel de abstracción superior al de las operaciones simples y la manipulación de variables.

Muchos modelos de aprendizaje automático pueden expresarse como formada por capas relativamente simples pero arregladas y apiladas. TensorFlow ofrece tanto un grupo de muchas capas comunes como formas sencillas para que usted escriba sus propias capas específicas de la aplicación, ya sea desde cero o como una composición de capas existentes.

TensorFlow viene con la API [Keras](https://keras.io) completa en el paquete tf.keras, y las capas Keras son muy útiles si desea construir sus propios modelos.


In [ ]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

Consulte la [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/layers) para ver la lista completa de capas preexistentes. Entre ellas se encuentran Dense (una capa totalmente conectada), Conv2D, LSTM, BatchNormalization, Dropout y muchas otras.

In [ ]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

In [ ]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

In [ ]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

## Implementar capas personalizadas

La mejor forma de implementar su propia capa es extender la clase tf.keras.Layer e implementar:

1. `__init__`, donde puede hacer toda la inicialización independiente de la entrada
2. `build`, donde usted sabe las formas de los tensores de entrada y puede hacer el resto de la inicialización por sí mismo.
3. `call`, donde usted realiza computación hacia adelante

Tenga en cuenta que no tiene que esperar hasta que se llame a `build`  para crear sus variables. También puede crearlas en `__init__`. Sin embargo, la ventaja de esperar a `build` es que le permite crear variables en una fase posterior según la forma de las entradas sobre las que operará la capa. Y a la inversa, crear variables en `__init__` significa que las formas necesarias para crear las variables tienen que definirse explícitamente.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [ ]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [ ]:
print([var.name for var in layer.trainable_variables])

El código en su conjunto es más fácil de leer y mantener si usa capas estándares siempre que sea posible, ya que los demás usuarios estarán familiarizados con el comportamiento de estas capas. ¡Considere la posibilidad de registrar una <a>incidencia en github</a> o, mejor aún, de enviarnos una solicitud de incorporación si desea usar una capa que no está en <code>tf.keras.layers</code>!

## Modelos: Composición de capas

En los modelos de aprendizaje automático, muchas cosas interesantes parecidas a las capas se implementan al hacer composiciones de capas existentes. Por ejemplo, cada bloque residual en una retícula es una composición de convoluciones, normalizaciones de lotes y un enlace. Puede anidar capas dentro de otras capas.

Normalmente se hace herencia de `keras.Model` cuando se necesita los métodos del modelo como `Model.fit`, `Model.evaluate` y `Model.save` (para más detalles, consulte [Capas y modelos personalizados de Keras](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

Otra característica de `keras.Model` (en lugar de `keras.layers.Layer`) es que no sólo realiza un seguimiento de las variables, sino que un `keras.Model` también sigue sus capas internas, lo que simplifica su inspección.

Por ejemplo, aquí hay un bloque ResNet:

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

La mayoría de las veces, sin embargo, los modelos que constan de muchas capas simplemente llaman a una capa tras otra. Esto puede hacerse con muy poco código usando `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Siguientes pasos

Ya puede volver al bloc de notas anterior y adaptar el ejemplo de regresión lineal usando capas y modelos para que esté mejor estructurado.